In [1]:
import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
from sagemaker import get_execution_role

role = get_execution_role()

region = boto3.Session().region_name

bucket='aws-logs-124182652801-us-east-2' # Replace with your s3 bucket name
prefix = 'aws-logs-124182652801-us-east-2/GitHubData' # Used as part of the path in the bucket where you store data
bucket_path = 'https://s3.console.aws.amazon.com/s3/buckets/aws-logs-124182652801-us-east-2/GitHubData/?region=us-east-2&tab=overview'.format(region,bucket) # The URL to access the bucket

In [2]:
import pickle, gzip, urllib.request, json
import numpy as np

# Load the dataset
urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz", "mnist.pkl.gz")
with gzip.open('mnist.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
print(train_set[0].shape)

(50000, 784)


In [8]:
%%time

import struct
import io
import csv
import boto3

# This is an example of taking unziped data and uploading each partition 

def convert_data():
    data_partitions = [('train', train_set), ('validation', valid_set), ('test', test_set)]
    for data_partition_name, data_partition in data_partitions:
        print('{}: {} {}'.format(data_partition_name, data_partition[0].shape, data_partition[1].shape))
        labels = [t.tolist() for t in data_partition[1]]
        features = [t.tolist() for t in data_partition[0]]
        
        if data_partition_name != 'test':
            examples = np.insert(features, 0, labels, axis=1)
        else:
            examples = features
        #print(examples[50000,:])
        
        
        np.savetxt('data.csv', examples, delimiter=',')
        
        
        
        key = "{}/{}/examples".format(prefix,data_partition_name)
        url = 's3://{}/{}'.format(bucket, key)
        boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_file('data.csv')
        print('Done writing to {}'.format(url))
        
convert_data()

train: (50000, 784) (50000,)
Done writing to s3://aws-logs-124182652801-us-east-2/GitHubData/train/examples
validation: (10000, 784) (10000,)
Done writing to s3://aws-logs-124182652801-us-east-2/GitHubData/validation/examples
test: (10000, 784) (10000,)
Done writing to s3://aws-logs-124182652801-us-east-2/GitHubData/test/examples
CPU times: user 50.4 s, sys: 4.71 s, total: 55.1 s
Wall time: 45.4 s


In [7]:
# !pip install --upgrade pip
!pip install google-cloud-bigquery

    100% |████████████████████████████████| 153kB 7.9MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 30.1MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 28.3MB/s ta 0:00:01
    100% |████████████████████████████████| 102kB 32.8MB/s a 0:00:01
  Running setup.py bdist_wheel for googleapis-common-protos ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/9e/3d/a2/1bec8bb7db80ab3216dbc33092bb7ccd0debfb8ba42b5668d5
Successfully built googleapis-common-protos
You are using pip version 10.0.1, however version 19.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
%load_ext google.cloud.bigquery

In [8]:
from google.cloud import bigquery
import os
import pandas as pd
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/ec2-user/SageMaker/CommitAnalysis/LamdaLabsProject-03b6fddae3e7.json"
client = bigquery.Client()

In [9]:
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")
table_ref = dataset_ref.table("commits")
table = client.get_table(table_ref)  # API call

# Load 50 rows from a table
rows = client.list_rows(table)
50 == table.num_rows

# Load the first 10 rows
rows = client.list_rows(table, max_results=10)
assert len(list(rows)) == 10

# Use the start index to load an arbitrary portion of the table
rows = client.list_rows(table, start_index=10, max_results=10)

commits_df = rows.to_dataframe()
commits_df

,commit,tree,parent,author,committer,subject,message,trailer,difference,difference_truncated,repo_name,encoding
0,a6f4f061df11a9d245a31cc038fb7e81b5e789d1,cab0a5d7df3b8871f66369b61ec0c4a3747adab1,"[b73ea76221d281c3026c46c730ccd26a889be5ff, 767...","{'name': 'Luke Dashjr', 'email': 'd0b35c280567...","{'name': 'Luke Dashjr', 'email': 'd0b35c280567...",Null-Merge commit '76712c1' into cg_merges_201...,Null-Merge commit '76712c1' into cg_merges_201...,"[{'key': 'Conflicts', 'value': '', 'email': No...",[],None,"[CryptoManiac/bfgminer, nwoolls/bfgminer, leon...",None
1,0a87a194d9716c3f23d82426ebba36b2b04bba15,ec250e766e6ff097df1f095f2a2356f3eade8956,[44406ea68e1a3a796decbcaeb32de9bafb362c0e],"{'name': 'Monty Taylor', 'email': '37c07a49ee3...","{'name': 'Jeremy Stanley', 'email': '98883b97e...",Add a floating IP if needed,"Add a floating IP if needed\n\nOn HP, when we ...","[{'key': 'Change-Id', 'value': 'I5c7ad3183d0b6...",[],None,"[citrix-openstack/project-config, anbangr/osci...",None
2,b14deab1cd1309ed769ff553c20be500ef3424fb,4b825dc642cb6eb9a060e54bf8d69288fbee4904,[125f5983abe31b47b8086dfe352c58cb209ebf75],"{'name': 'Иванов Денис Игоревич', 'email': 'ff...","{'name': 'Иванов Денис Игоревич', 'email': 'ff...",[NA] service bus receiver refactorings,[NA] service bus receiver refactorings\n\ngit-...,"[{'key': 'git-tfs-id', 'value': '[http://uk-tf...",[],None,[2gis/nuclear-aggregates-layer],None
3,6c256f500c7b31eff98f455c211703d4c4f8f0a9,2632a7248918151aa96008d0929ce5e6c42cebc8,"[15b23a91278b7854bda3158f1e3a5632eabafe1d, 3ed...","{'name': 'Chris Banes', 'email': '2db0fc324336...","{'name': 'android-build-merger', 'email': '00d...","Merge \\\\""Fix use of API 19+ FrameLayout LP c...","Merge \\\\""Fix use of API 19+ FrameLayout LP c...","[{'key': 'Change-Id', 'value': 'I93a84b338f35e...",[],None,[aosp-mirror/platform_frameworks_support],None
4,0472a5efeacae02d8bca4e44e0ced892785889dc,4b825dc642cb6eb9a060e54bf8d69288fbee4904,[d2b1810567fdf53b0e5699099029cdd4a9d7553c],"{'name': 'Rik Faith', 'email': '07479feb7f9378...","{'name': 'Rik Faith', 'email': '07479feb7f9378...",Added simple test suite,Added simple test suite\n\n\nSKIPPED:\n\tlibma...,"[{'key': 'SKIPPED', 'value': '', 'email': None}]",[],None,[cheusov/dictd],None
5,1e31d6bb52ef4046b1a419c1797e2dc99ca44138,8727a1fb203f9fe9c5f9941c9501facda19fa36a,[a9cb99e379ee9caf913ad5e4b226e88654ec761e],"{'name': 'Nathalie Lammelin', 'email': '2d8828...","{'name': 'Nathalie Lammelin', 'email': '2d8828...",closed #4073 - Fixed XmlCopy step when source ...,closed #4073 - Fixed XmlCopy step when source ...,"[{'key': 'Former-commit-id', 'value': '42ad095...",[],None,[convertigo/convertigo-webapp],None
6,69fb29c5a939e777f59221a521ca2d35ad5af69a,0daf946a8052a013250700af93121b6efb157e8b,[1716cdf190bcfa205db644fa9c8b25efb64527e4],"{'name': 't_voss', 'email': 'e64a528f711f6911b...","{'name': 't_voss', 'email': 'e64a528f711f6911b...",[Shark 3.0] ADDED: Abstract classes for multi-...,[Shark 3.0] ADDED: Abstract classes for multi-...,"[{'key': 'ADDED', 'value': 'AbstractBoxConstra...",[],None,[CptCapy/Programming_Shark],None
7,ac6db99daa9fa29cdd44302832199dc9847e6bd7,c64dfb2c94d4f43a4db503ac8bdb37559250c03b,[51c805ce28f394b5321814ed2332d8bd2c5bd399],"{'name': 'mezger', 'email': 'mezger', 'time_se...","{'name': 'mezger', 'email': 'mezger', 'time_se...",printer modification,printer modification\n\n\nSKIPPED:\n\tJava/caR...,"[{'key': 'SKIPPED', 'value': '', 'email': None}]",[],None,"[dassjosh/caqtdm, caqtdm/caqtdm]",None
8,86a4dc24843573d280f5db9e96c1c66092db3c74,d3b704365b686401124eb6618f5c2a907d12ce8e,[0a4d124f66a95815c0520bae564a82a2853c9047],"{'name': 'juhosg', 'email': '0b85b0feb94c9a44e...","{'name': 'juhosg', 'email': '0b85b0feb94c9a44e...",packages/ntripcaster: fix whitespaces,packages/ntripcaster: fix whitespaces\n\nSigne...,"[{'key': 'Signed-off-by', 'value': 'Dirk Neuki...",[],None,[qqyfly/attitude_adjustment],None
9,75ee3b3a23e29bb9553332af762f64549e1231db,b572c0163b511c5e66e0c2083acd659ad8af105f

In [19]:
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  aws-logs-124182652801-us-east-2


In [17]:
import boto3
s3 = boto3.Session().resource('s3')
bucket = 'github-analysis-test' #bucket name
key = 'DataSets/test.csv' # filepath in s3
doc = commits_df.to_csv('test.csv') # your local file

In [18]:
#Upload file to s3 bucket 
s3.Bucket(bucket).Object(key).upload_file('test.csv')

In [43]:
#Increase max columns to 999 to see all columns 
pd.options.display.max_columns = 999

In [ ]:
dataset_ref = client.dataset("samples", project="bigquery-public-data")
table_ref = dataset_ref.table("github_timeline")
table = client.get_table(table_ref)  # API call

# Load 50 rows from a table
rows = client.list_rows(table)
50 == table.num_rows

# Load the first 10 rows
rows = client.list_rows(table, max_results=50)
assert len(list(rows)) == 50

# Use the start index to load an arbitrary portion of the table
rows = client.list_rows(table, start_index=10, max_results=50)

ghtimeline_df = rows.to_dataframe()
ghtimeline_df

In [ ]:
dataset_ref = client.dataset("month", project="githubarchive")
table_ref = dataset_ref.table("201102")
table = client.get_table(table_ref)  # API call

# Load 50 rows from a table
rows = client.list_rows(table)
50 == table.num_rows

# Load the first 10 rows
rows = client.list_rows(table, max_results=50)
assert len(list(rows)) == 50

# Use the start index to load an arbitrary portion of the table
rows = client.list_rows(table, start_index=10, max_results=50)

ghmonth_df = rows.to_dataframe()
ghmonth_df

In [47]:
print(ghmonth_df.columns)

Index(['type', 'public', 'payload', 'repo', 'actor', 'org', 'created_at', 'id',
       'other'],
      dtype='object')


In [52]:
ghmonth_df.created_at[0]

Timestamp('2011-02-19 12:49:04+0000', tz='UTC')

In [54]:
ghmonth_df.head()

,type,public,payload,repo,actor,org,created_at,id,other
0,ForkEvent,True,"{""repo"":""hermesEOL/fail0verflow2"",""actor"":""Fro...","{'id': None, 'name': '/', 'url': 'https://api....","{'id': 626847, 'login': 'Frogggy', 'gravatar_i...","{'id': None, 'login': None, 'gravatar_id': Non...",2011-02-19 12:49:04+00:00,1141283335,None
1,ForkEvent,True,"{""repo"":""symfony/symfony-sandbox"",""actor"":""mea...","{'id': None, 'name': '/', 'url': 'https://api....","{'id': 632025, 'login': 'meandmymonkey', 'grav...","{'id': None, 'login': None, 'gravatar_id': Non...",2011-02-22 22:27:04+00:00,1147410078,None
2,ForkEvent,True,"{""repo"":""hermesEOL/fail0verflow"",""actor"":""guym...","{'id': None, 'name': '/', 'url': 'https://api....","{'id': 192004, 'login': 'guymann', 'gravatar_i...","{'id': None, 'login': None, 'gravatar_id': Non...",2011-02-15 03:38:46+00:00,1132505427,None
3,ForkEvent,True,"{""repo"":""rupa/z"",""actor"":""xl42ii"",""forkee"":140...","{'id': 236571, 'name': 'rupa/z', 'url': 'https...","{'id': 439304, 'login': 'xl42ii', 'gravatar_id...","{'id': None, 'login': None, 'gravatar_id': Non...",2011-02-25 01:52:03+00:00,1151856530,None
4,ForkEvent,True,"{""repo"":""be9/acl9"",""actor"":""mariojorge"",""forke...","{'id': 100181, 'name': 'be9/acl9', 'url': 'htt...","{'id': 231730, 'login': 'mariojorge', 'gravata...","{'id': None, 'login': None, 'gravatar_id': Non...",2011-02-16 12:20:24+00:00,1135061844,None


In [59]:
ghmonth_df.payload[0]

'{"repo":"hermesEOL/fail0verflow2","actor":"Frogggy","forkee":1386047,"actor_gravatar":"f404af943a6bfbdd92d5715b0a8e3dd4"}'

In [60]:
ghmonth_df.repo[0]

{'id': None, 'name': '/', 'url': 'https://api.github.dev/repos//'}

In [61]:
ghmonth_df.actor[0]

{'id': 626847,
 'login': 'Frogggy',
 'gravatar_id': 'f404af943a6bfbdd92d5715b0a8e3dd4',
 'avatar_url': 'https://secure.gravatar.com/avatar/f404af943a6bfbdd92d5715b0a8e3dd4?d=http://github.dev%2Fimages%2Fgravatars%2Fgravatar-user-420.png',
 'url': 'https://api.github.dev/users/Frogggy'}

In [62]:
ghmonth_df.org[0]

{'id': None,
 'login': None,
 'gravatar_id': None,
 'avatar_url': None,
 'url': None}

In [63]:
ghmonth_df.created_at[0]

Timestamp('2011-02-19 12:49:04+0000', tz='UTC')